In [ ]:
# Notebook from https://medium.com/@thakermadhav/build-your-own-rag-with-mistral-7b-and-langchain-97d0c92fa146
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7
!pip install -q scipy langchain transformers playwright html2text sentence_transformers faiss-gpu
!pip install -q --upgrade git+https://github.com/huggingface/transformers

In [ ]:
!playwright install > /dev/null
!playwright install-deps > /dev/null

In [ ]:
!pip install -U tokenizers

In [ ]:
import os
import torch
from transformers import (
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  pipeline
)

from transformers import BitsAndBytesConfig




from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain


#################################################################
# Tokenizer
#################################################################

model_name="mistralai/Mistral-7B-Instruct-v0.1"
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    cache_dir=".",
)
#load_in_4bits=True)


In [123]:
model.config.pad_token_id = tokenizer.pad_token_id

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [124]:
import arrow
import pandas as pd
PARQUET_FILE="Accords/echantillon_public_Mathilde.parquet"

In [125]:
df_init=pd.read_parquet(PARQUET_FILE)

In [126]:
prompt_template = """

### [ROLE] Role: Vous êtes un expert juridique des accords d'entreprise sans compétence de programmation informatique, seulement la lecture et la compréhension des textes. Tu ne connais aucun langage de programmation, ni les expressions régulières.


### [INST] Instructions: 

1. Votre travail consiste à peupler et mettre à jour une base des données nécessaires pour réaliser des études économétriques poussées sur le télétravail.
        Pour cela, vous devez prendre le texte d'un accord d'entreprise dans le document suivant, dans lesquelles figurent les informations à jour que l'on vous fourni, et d'extraire précisément l'information spécifique dont vous avez besoin.
        Vous ne devez extraire que le nombre, et uniquement le nombre. Le nombre peut être écrit en lettres ou en chiffres. (exemple : deux ou 2 )
        Si vous ne trouvez pas la réponse dans le texte vous devez le dire et ne pas chercher à en fournir un autre nombre.
        Avant de répondre, vérifier que la réponse se trouve bien dans le texte indiqué.
        
2. Vous devez respecter les règles suivantes : 
        2.A - Déconnecter les liens entre les périodicités (semaine, mois, trimestre, année)! Faire comme si les semaines, les mois, les trimestres et les années sont des concepts indépendants : dans un mois, il y a donc 0 semaine ; dans un trimestre, 0 mois et 0 semaine; dans une année, 0 trimestre, 0 mois et 0 semaine !
        2.B - Ne faites aucune supposition ni d'hypothèses : déconnectez le module de raisonnement !
        2.C - Ne faites aucun calcul de proratisation : par exemple, s'il y a 10 jours de télétravail autorisés par an, alors on ne peut pas déduire le nombre de jour de télétravail autorisés par semaine.
        2.D - Il n'y a pas de lien entre l'effectif d'un service et le nombre de jour de télétravail, donc ne pas déduire un nombre de jour de télétravail !
        2.E - Si le document mentionne positivement le Télétravail à 100%, ou le full-remote, ET qu'il n'y a pas de jour de présence minimum sur site, alors 5 jours de télétravail est possible par semaine !
        2.F - Si le document mentionne par exemple trois jours de présence sur site et sans autre mention du nombre de jours de télétravail autorisé, alors tu ne peux pas donner de nombre de jour de télétravail par semaine a priori !
        2.G - Il y a cinq jours ouvrés dans une semaine, donc un nombre de jour de télétravail par semaine ne peut pas être strictement supérieur à cinq.
        2.H - Si tu donnes un nombre réel, ne donne pas la fraction mais un arrondi à l'entier inférieur !
        2.I - Si le nombre de jour de télétravail annuel est inférieur à 52, ce nombre de jour de télétravail concerne du télétravail occasionnel. S'il est supérieur ou égal à 52, ce nombre concerne du télétravail régulier.
        2.J - Si le nombre de jour de télétravail trimestriel est inférieur à 13, ce nombre de jour de télétravail concerne du télétravail occasionnel. S'il est supérieur ou égal à 13, ce nombre concerne du télétravail régulier.
        2.K - Si le nombre de jour de télétravail mensuelle est inférieur à 4, ce nombre de jour de télétravail concerne du télétravail occasionnel. S'il est supérieur ou égal à 4, ce nombre concerne du télétravail régulier.

3. Vous devez structurez votre réponse comme suit :
        * Mon extraction de deux paragraphes max pertinents provenant du document à analyser qui répondent à la question (limité chacun à deux phrases max):
        * Ma réponse :
        * Mon raisonnenement :
        
### Document à analyser: 
{context}

### QUESTION:
{question} [/INST]
"""

In [127]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [128]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
import warnings
warnings.filterwarnings('ignore')
import re
def guess_reponse_booleenne(reponse):
    reponse_booleenne=None
    if reponse["text"].startswith("\nOui") or reponse["text"].startswith("Oui") :
        reponse_booleenne=1
    elif reponse["text"].startswith("\nNon") or reponse["text"].startswith("Non") :
        reponse_booleenne=0
    return reponse_booleenne

def guess_reponse_durée(reponse):
    reponse_booleenne=None
    if "indéterminée" in reponse["text"].lower() :
        reponse_booleenne=0
    elif "déterminée" in reponse["text"].lower():
        reponse_booleenne=1
    return reponse_booleenne

def guess_reponse_nombre(reponse,pattern="nombre"):
    reponse_nombre=None
    REGEX=rf".*{pattern} ?= ?(\d+)"
    numbers=re.findall(REGEX,reponse["text"])
    if numbers:
        reponse_nombre=numbers[0]
    return reponse_nombre

In [129]:
Q_DUREE="S'il est fait mention d'une durée de l'accord, est-ce que l'accord est à durée déterminée ou à durée indéterminée ? déterminée=1 ou indéterminée=0 ou NA?"
Q_REVERS="S'il est fait mention d'une clause et d'un article de réversibilité, est-ce que l'accord a une clause ou un article de réversibilité ? Oui=1 ou non=0 ou NA?"
Q_ADAPT="S'il est fait mention d'une période d'adaptation, est-ce que l'accord comprend une période d'adaptation ? Oui=1 ou non=0 ou NA?"
Q_TTREG="Est-ce qu'au moins un paragraphe mentionne du télétravail régulier ?"
Q_TTOCA="Est-ce qu'au moins un paragraphe mentionne du télétravail occasionnel ?"
Q_TTEXC="Est-ce qu'au moins un paragraphe mentionne du télétravail exceptionnel ?"
Q_DISPOSPERQTHENCEINTE="Est-ce qu'au moins un paragraphe mentionne des dispositifs spéciaux pour les travailleurs en situation de handicap, les femmes enceintes, les séniors ou les personnes présentant des fragilités ou des problèmes de santé ?"
Q_MONTANT_CAVIARDE="Est-ce qu'au moins un paragraphe mentionne un nombre sans le donner exactement, mais sous forme illisible ?"
Q_TELETRAVAIL_FLEX_SANS_LIMITE="Est-ce qu'au moins un paragraphe suggère une liberté totale dans le choix des jours de télétravail, hormis quelques contraintes de présence ?"
Q_TT_REG_NOMBRE_FORMULE=Q_TTREG+"Si oui, combien de formules (exemples de formule : si temps plein et temps partiel = 2, si choix entre 1 à 3 jours de télétravail régulier = 3) sont proposées pour du télétravail régulier ? Retourne à la fin 'nombre=max(tes réponses)'"
PERIODICITE={"MOIS":"mois","TRIM":"trimestre","ANNEE":"année", "SEM":"semaine"}
PERIODICITE2={"JOUR":"jour","MOIS":"mois","TRIM":"trimestre","ANNEE":"année", "SEM":"semaine"}
Q_COMPE="Est-ce qu'au moins un paragraphe mentionne une indemnité spécialement pour l'achat d'équipements ?"
Q_COMPE_NOMBRE=Q_COMPE+"Si oui, sélectionne seulement les paragraphes mentionnant une indemnité spécialement pour l'achat d'équipements et s'il existe donne le montant lié de l'indemnité d'équipement. Si oui, retourne à la fin 'nombre=max(ta réponse)'"
Q_EQUIPEMENT="Est-ce qu'au moins un paragraphe mentionne un équipement de télétravail fourni ?"
Q_COMPS="Est-ce qu'au moins un paragraphe mentionne explicitement une indemnité de sujétion (dans le cas où l'employeur impose le télétravail, ce dernier indemnise l'employé) ?"
Q_COMPS_NOMBRE=Q_COMPS+"Si oui, sélectionne seulement les paragraphes mentionnant une indemnité de sujétion dans le cas où l'employeur impose le télétravail et s'il existe donne le montant lié de l'indemnité de sujétion dans le cas où l'employeur impose le télétravail. Si oui, retourne à la fin 'nombre=max(ta réponse)'"

In [130]:
def process_model(rag_chain,num_dossier):
    dict_reponse=dict()
    dict_reponse["num_dossier"]=num_dossier
    text=""
    reponse= rag_chain.invoke(Q_DISPOSPERQTHENCEINTE)
    text+="Q_DISPOSPERQTHENCEINTE:"+reponse["text"] + "\n"
    dict_reponse["DISPOSPERQTHENCEINTE"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_MONTANT_CAVIARDE)
    text+="Q_MONTANT_CAVIARDE:"+reponse["text"] + "\n"
    dict_reponse["MONTANT_CAVIARDE"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TELETRAVAIL_FLEX_SANS_LIMITE)
    text+="Q_TELETRAVAIL_FLEX_SANS_LIMITE:"+reponse["text"] + "\n"
    dict_reponse["TELETRAVAIL_FLEX_SANS_LIMITE"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TT_REG_NOMBRE_FORMULE)
    text+="Q_TT_REG_NOMBRE_FORMULE:"+reponse["text"] + "\n"
    dict_reponse["TT_REG_NOMBRE_FORMULE"]=guess_reponse_nombre(reponse)
    reponse= rag_chain.invoke(Q_DUREE)
    text+="Q_DUREE:"+reponse["text"] + "\n"
    dict_reponse["DUREE"]=guess_reponse_durée(reponse)
    reponse= rag_chain.invoke(Q_REVERS)
    text+="Q_REVERS:"+reponse["text"] + "\n"
    dict_reponse["REVERS"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_ADAPT)
    text+="Q_ADAPT:"+reponse["text"] + "\n"
    dict_reponse["ADAPT"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TTREG)
    text+="Q_TTREG:"+reponse["text"] + "\n"
    dict_reponse["TTREG"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TTOCA)
    text+="Q_TTOCA:"+reponse["text"] + "\n"
    dict_reponse["TTOCA"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TTEXC)
    text+="Q_TTEXC:"+reponse["text"] + "\n"
    dict_reponse["TTEXC"]=guess_reponse_booleenne(reponse)
    for (k,v) in PERIODICITE.items():
        Q_TT=f"Hors jours supplémentaires ou exceptionnels, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail maximum autorisé explicitement exprimé par {v} ?"
        reponse= rag_chain.invoke(Q_TT)
        text+=f"Q_TT{k}:"+reponse["text"] + "\n"
        dict_reponse[f"TT{k}"]=guess_reponse_booleenne(reponse)
        #Q_TT_NOMBRE=f"Extrait le nombre de jour de télétravail maximum autorisé explicitement exprimé par {v} seulement à partir du texte suivant : {text_reponse}"
        Q_TT_NOMBRE=Q_TT+f"Si oui, quel est le nombre de jour de télétravail supplémentaires explicitement exprimé et nécessairement et seulement exprimé par {v} pour des dispositions spéciales. Si oui, retourne à la fin 'nombre=max(tes réponses)'"
        reponse= rag_chain.invoke(Q_TT_NOMBRE)
        text+=f"Q_TT{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"TT{k}_NOMBRE"]=guess_reponse_nombre(reponse)
    for (k,v) in PERIODICITE.items():
        Q_TTOCA_PERIOD=f"Dans le cadre du télétravail occasionnel, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail maximum autorisé explicitement exprimé par {v} ?"
        reponse= rag_chain.invoke(Q_TTOCA_PERIOD)
        text+=f"TTOCA{k}:"+reponse["text"] + "\n"
        dict_reponse[f"TTOCA{k}"]=guess_reponse_booleenne(reponse)
        Q_TTOCA_NOMBRE=Q_TTOCA_PERIOD+f"Si oui, quel est le nombre de jour de télétravail occasionnel explicitement exprimé et nécessairement et seulement exprimé par {v}. Si oui, retourne à la fin 'nombre=max(tes réponses)'"
        reponse= rag_chain.invoke(Q_TTOCA_NOMBRE)
        text+=f"Q_TTOCA_{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"TTOCA_{k}_NOMBRE"]=guess_reponse_nombre(reponse)
    for (k,v) in PERIODICITE.items():
        Q_PRESJOUR=f"Est-ce qu'au moins un paragraphe mentionne un nombre de jour de présence obligatoire sur site minimum explicitement exprimé par {v} ?"
        reponse= rag_chain.invoke(Q_PRESJOUR)
        text+=f"Q_PRESJOUR{k}:"+reponse["text"] + "\n"
        dict_reponse[f"TTPRESJOUR{k}"]=guess_reponse_booleenne(reponse)
        Q_PRESJOUR_NOMBRE=Q_PRESJOUR+f"Si oui, sélectionne seulement les paragraphes mentionnant un nombre de jour de présence obligatoire sur site minimum explicitement exprimé par {v} et s'il existe donne le nombre de jour de présence obligatoire sur site minimum explicitement exprimé par {v}. Si oui, retourne à la fin 'nombre=max(tes réponses)'"
        reponse= rag_chain.invoke(Q_PRESJOUR_NOMBRE)
        text+=f"Q_PRESJOUR{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"PRESJOUR{k}_NOMBRE"]=guess_reponse_nombre(reponse)
    for (k,v) in PERIODICITE2.items():
        Q_COMP=f"Hors indemnité d'équipement, est-ce qu'au moins un paragraphe mentionne une indemnité forfaitaire explicitement exprimée par {v} ?"
        reponse= rag_chain.invoke(Q_COMP)
        text+=f"Q_COMP{k}:"+reponse["text"] + "\n"
        dict_reponse[f"COMP{k}"]=guess_reponse_booleenne(reponse)
        Q_COMP_NOMBRE=Q_COMP+f"Si oui, sélectionne en interne seulement les paragraphes mentionnant une indemnité forfaitaire explicitement exprimé par {v}, puis s'il existe donne le montant explicitement exprimé par {v}. Si oui, retourne à la fin 'nombre=max(ta réponse)'"
        reponse= rag_chain.invoke(Q_COMP_NOMBRE)
        text+=f"Q_COMP{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"COMP{k}_NOMBRE"]=guess_reponse_nombre(reponse)
        if "MOIS" in k:
            Q_COMP_NOMBRE_BASE=Q_COMP+f"Si oui, sélectionne en interne seulement les paragraphes mentionnant une indemnité forfaitaire explicitement exprimé par {v}, puis s'il existe plusieurs montants explicitement exprimé par {v}, par nombre de jour. Si oui, retourne à la fin 'nombre=base_mensuelle_pour_un_jour_de_teletravail(ta réponse)'"
            reponse= rag_chain.invoke(Q_COMP_NOMBRE_BASE)
            text+=f"Q_COMP{k}_BASE_NOMBRE:"+reponse["text"] + "\n"
            dict_reponse[f"COMP{k}_BASE_NOMBRE"]=guess_reponse_nombre(reponse)
    reponse= rag_chain.invoke(Q_COMPE)
    text+="Q_COMPE:"+reponse["text"] + "\n"
    dict_reponse["COMPE"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_COMPE_NOMBRE)
    text+="Q_COMPE_NOMBRE:"+reponse["text"] + "\n"
    dict_reponse["COMPE_NOMBRE"]=guess_reponse_nombre(reponse)
    reponse= rag_chain.invoke(Q_EQUIPEMENT)
    text+="Q_EQUIPEMENT:"+reponse["text"] + "\n"
    dict_reponse["EQUIPEMENT"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_COMPS)
    text+="Q_COMPS:"+reponse["text"] + "\n"
    dict_reponse["COMPS"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_COMPS_NOMBRE)
    text+="Q_COMPS_NOMBRE:"+reponse["text"] + "\n"
    dict_reponse["COMPS_NOMBRE"]=guess_reponse_nombre(reponse)
    return dict_reponse,text

In [131]:
PERIODICITE={ "SEM":"semaine"}
EXEMPLES= "Le télétravail pour le salarié à temps complet ou titulaire d’un forfait annuel 218 jours ne pourra être exercé que pour un maximum de 13 jours (soit 26 demi-journées) par trimestre civil dans la limite de 1 jour (soit 2 demi-journées) par semaine de 5 jours travaillés, non reportables, soit une enveloppe de 52 jours (soit 104 demi-journées) par année civile. => 1 jour de télétravail par semaine."
EXEMPLES+= " Un maximum de 2 jours par semaine, et de 2 demi-journées pouvant être cumulées sur une même journée au cours de la même semaine. => 3 jours de télétravail par semaine car 3 = (2+2*0.5)."
EXEMPLES+= " Une limite haute de 10 jours de télétravail, par mois civil, est instituée. => Information à ignorer."
EXEMPLES+= " Les salariés devront être présents au minimum 2 jours par semaine sur leur lieu de travail habituel. => 2 jour sur site par semaine."
EXEMPLES+= " Le télétravail pour le salarié à temps complet ou titulaire d’un forfait annuel 218 jours ne pourra être exercé que pour un maximum de 13 jours (soit 26 demi-journées) par trimestre civil dans la limite de 1 jour (soit 2 demi-journées) par semaine de 5 jours travaillés, non reportables, soit une enveloppe de 52 jours (soit 104 demi-journées) par année civile. => 1 jour de télétravail par semaine."
EXEMPLES+= " Le nombre de jour de télétravail est à discuter avec le manager. => Information à ignorer."
EXEMPLES+= " Les 4 journées de télétravail mensuelles autorisées et 1 jour de télétravail fixe par semaine calendaire => 1 jour de télétravail par semaine."

def process_model(rag_chain,num_dossier):
    dict_reponse=dict()
    dict_reponse["num_dossier"]=num_dossier
    text=""
    for (k,v) in PERIODICITE.items():
        Q_TT_NOMBRE=f"En dehors des instructions et hors télétravail occasionnel ou ponctuel, est-ce qu'un paragraphe du document comporte le mot-clé {v} ou une notion de fréquence par {v} et concerne le nombre de jour de télétravail autorisé (ne compte pas les paragraphes qui relatent d'indemnisation par jour de télétravail, ni le télétravail ponctuel ou occasionnel)?"
        Q_TT_NOMBRE+=f" Si oui, depuis les informations du document, jusqu'à combien de jour possible (ou limite) de télétravail un employé/travailleur/salarié/collaborateur peut-il télétravailler régulièrement, jour exprimé seulement par {v} et explicitement dans un des paragraphes (attention, ne pas confondre avec les jours de présence obligatoire sur site)? "
        Q_TT_NOMBRE+=f" Si tu as plusieurs réponses parmi [ 0 , 0.5 , 1 , 1.5 , 2 , 2.5 , 3 , 3.5 , 4 , 4.5 , 5 ], affecte le maximum des jours de télétravail par {v} de ces éventuelles réponses dans une variable 'nombrejourdeteletravailmaxpar{v}'. Si tu ne trouves pas de valeur ou si les informations ne permettent pas de déterminer cette valeur, retourne nombrejourdeteletravailmaxpar{v} à 0"
        Q_TT_NOMBRE+=f" Si le télétravail à 100% ou le full-remote est autorisé ou positivement évoqué ET qu'il n'y a pas de jour de présence minimum sur site, alors affecte à la variable nombrejourdeteletravailmaxpar{v} la valeur 5."
        Q_TT_NOMBRE+=f" INSTRUCTIONS SUPPLEMENTAIRES : "
        Q_TT_NOMBRE+=f" Pour cette tâche, ignore les informations concernant les répartitions d'effectif sur site, les négociations avec les managers, les définitions du télétravail."
        Q_TT_NOMBRE+=f" En l'absence de précisions explicites du nombre de jour de télétravail par {v}, affecte impérativement à la variable nombrejourdeteletravailmaxpar{v} la valeur 0."
        Q_TT_NOMBRE+=f" 0 est la valeur la plus basse du nombre de jour de télétravail maximum par {v}. "
        Q_TT_NOMBRE+=f" S'il y a des jours de présence obligatoire sur site, retourne également une variable supplémentaire 'nombrejoursursitepar{v}' avec la valeur indiquée"
        Q_TT_NOMBRE+=f" Ta réponse devra impérativement comporter la chaîne de caractère 'nombrejourdeteletravailmaxpar{v}=' suivi du nombre demandé."
        Q_TT_NOMBRE+=f" Voici quelques exemples pour vous aider : {EXEMPLES}"
        reponse= rag_chain.invoke(Q_TT_NOMBRE)
        text+=f"Q_TT{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"TT{k}_NOMBRE"]=guess_reponse_nombre(reponse,pattern=f"nombrejourdeteletravailmaxpar{v}")
    return dict_reponse,text

In [132]:
VAR_TRANSLATION={'DISPOSPERQTHENCEINTE': "dispositif_special_rqth_maternite_senior",
 'MONTANT_CAVIARDE': "montant_caviarde",
 'TELETRAVAIL_FLEX_SANS_LIMITE': "teletravail_regulier_flexible_sans_limite",
 'TT_REG_NOMBRE_FORMULE': "nombre_formules_teletravail_regulier",
 'DUREE': "duree_application",
 'REVERS': "presence_clause_reversibilite",
 'ADAPT': "periode_adaptation",
 'TTREG': "teletravail_regulier",
 'TTOCA': "teletravail_occasionnel",
 'TTEXC': "teletravail_exceptionnel",
 'TTMOIS': "mention_teletravail_par_mois",
 'TTMOIS_NOMBRE': "nombre_jours_teletravail_mois",
 'TTTRIM': "mention_teletravail_par_trimestre",
 'TTTRIM_NOMBRE': "nombre_jours_teletravail_trimestre",
 'TTANNEE': "mention_teletravail_par_annuel",
 'TTANNEE_NOMBRE': "nombre_jours_teletravail_annuel",
 'TTSEM': "mention_teletravail_par_semaine",
 'TTSEM_NOMBRE': "nombre_jours_teletravail_semaine",
 'TTOCAMOIS': "occas_mention_teletravail_par_mois",
 'TTOCA_MOIS_NOMBRE': "occas_nombre_jours_teletravail_mois",
 'TTOCATRIM': "occas_mention_teletravail_par_trimestre",
 'TTOCA_TRIM_NOMBRE': "occas_nombre_jours_teletravail_trimestre",
 'TTOCAANNEE': "occas_mention_teletravail_par_annuel",
 'TTOCA_ANNEE_NOMBRE': "occas_nombre_jours_teletravail_annuel",
 'TTOCASEM': "occas_mention_teletravail_par_semaine",
 'TTOCA_SEM_NOMBRE': "occas_nombre_jours_teletravail_semaine",
 'TTPRESJOURMOIS': "mention_jour_presence_par_mois",
 'PRESJOURMOIS_NOMBRE': "nombre_jour_presence_par_mois",
 'TTPRESJOURTRIM': "mention_jour_presence_par_trimestre",
 'PRESJOURTRIM_NOMBRE': "nombre_jour_presence_par_trimestre",
 'TTPRESJOURANNEE': "mention_jour_presence_par_annuel",
 'PRESJOURANNEE_NOMBRE': "nombre_jour_presence_par_annuel",
 'TTPRESJOURSEM': "mention_jour_presence_par_semaine",
 'PRESJOURSEM_NOMBRE': "nombre_jour_presence_par_semaine",
 'COMPJOUR': "mention_indemnisation_journaliere",
 'COMPJOUR_NOMBRE': "indemnisation_journaliere",
 'COMPMOIS': "mention_indemnisation_mensuelle",
 'COMPMOIS_NOMBRE': "indemnisation_mensuelle",
 'COMPMOIS_BASE_NOMBRE': "indemnisation_base_mensuelle_pour_un_jour_par_semaine",
 'COMPTRIM': "mention_indemnisation_trimestrielle",
 'COMPTRIM_NOMBRE': "indemnisation_trimestrielle",
 'COMPANNEE': "mention_indemnisation_annuelle",
 'COMPANNEE_NOMBRE': "indemnisation_annuelle",
 'COMPSEM': "mention_indemnisation_semaine",
 'COMPSEM_NOMBRE': "indemnisation_semaine",
 'COMPE': "mention_indemnisation_equipement",
 'COMPE_NOMBRE': "indemnisation_equipement",
 'COMPS': "mention_indemnite_sujetion",
 'COMPS_NOMBRE': "montant_indemnite_sujetion",
 'EQUIPEMENT': "equipement_fourni"}

# Boucle

In [133]:
import glob
PARQUET_FILE_COMPARE="Accords/Donnees_Mathilde_Pesenti_TT_Pour_DEE_v2_def.parquet"
for (index,(num_dossier,accord)) in df_init.iterrows():
    print(index,num_dossier)
    with open(f"{num_dossier}.txt","w", encoding="utf-8") as file:
        file.write(accord)

    loader = TextLoader(f"{num_dossier}.txt", encoding='utf8')
    docs=loader.load()
    
    # Chunk text
    text_splitter = CharacterTextSplitter(chunk_size=5000, 
                                          chunk_overlap=100)
    chunked_documents = text_splitter.split_documents(docs)
    
    # Load chunked documents into the FAISS index
    db = FAISS.from_documents(chunked_documents, 
                              HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))
    
    retriever = db.as_retriever()
    
    rag_chain = ( 
     {"context": retriever, "question": RunnablePassthrough()}
        | llm_chain
    )
    dict_reponse,text=process_model(rag_chain,num_dossier)
    with open(f"{num_dossier}_response.txt","w", encoding="utf-8") as file:
        file.write(text)
        print(text)
    df2=pd.DataFrame.from_dict(dict_reponse,orient="index")
    df2=df2.transpose().rename(columns=VAR_TRANSLATION)
    df2.to_csv(f"{num_dossier}.csv")
    df2.to_parquet(f"{num_dossier}.parquet")

0 T09221025781


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q_TTSEM_NOMBRE:Extraction de paragraphes pertinents:

* "Le télétravail doit être ponctuel : Il ne pourra être exercé que pour un maximum de 10 jours par an. La répartition du temps de travail s’opère par journée ou demi-journée."

Ma réponse :
nombrejourdeteletravailmaxparsemaine=0

Mon raisonnement :
Le document ne contient aucune information explicite sur le nombre de jours de télétravail par semaine, sauf une limite annuelle de 10 jours. Par conséquent, je ne peux pas affecter une valeur supérieure à 0 à la variable nombrejourdeteletravailmaxparsemaine.

1 T03821007340


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q_TTSEM_NOMBRE:
Extraction de deux paragraphes pertinents du document :

* "En raison de la crise sanitaire de 2020 et de ses conséquences en matière économique et d’activité, la société ARaymond France a été contrainte de recourir au dispositif d’activité partielle."
* "Durant le premier confinement général, et en application des dispositions légales applicables en cas d’épidémie ou de pandémie, le recours au télétravail a été généralisé pour tous les salariés en activité dont le poste et/ou leur mission permettait ledit télétravail. Depuis, le télétravail est resté la règle et la présence sur site l’exception pour tous les salariés pouvant avoir recours à ce mode d’organisation du travail."

Ma réponse :

nombrejourdeteletravailmaxparsemaine=0

Mon raisonnement :

Le document ne contient aucune information explicite sur le nombre de jours de télétravail par semaine. Par conséquent, je dois affecter impérativement à la variable nombrejourdeteletravailmaxparsemaine la valeur 0.

2 T092

...

Q_TTSEM_NOMBRE:Mon extraction de deux paragraphes max pertinents provenant du document à analyser qui répondent à la question :

* "Le collaborateur cadre, en télétravail, pourra organiser sa journée en respectant la réglementation sur le temps de travail et les règles définies dans l’accord OATT en vigueur. [...] Le télétravail est limité de telle sorte que le collaborateur cadre soit présent dans l’entreprise au moins trois jours par semaine."
* "Organisation du travail pour les collaborateurs commerciaux \n\nEn raison de la particularité du poste de commercial, l’activité s’exerce essentiellement hors des locaux de l’entreprise. Les collaborateurs commerciaux devront être présents sur site au minimum quatre jours par mois.\xa0"

Ma réponse :
nombrejourdeteletravailmaxparsemaine=1

Mon raisonnement :
Le premier paragraphe indique que les collaborateurs cadres doivent être présents dans l'entreprise au moins trois jours par semaine. Par déduction, ils peuvent télétravailler au maximum

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q_TTSEM_NOMBRE:Extrait pertinent du document :
"Il a été convenu, par le présent accord, d’augmenter le nombre de jours de travail à **deux jours par semaine** permettant aux salariés concernés de recourir au télétravail en application des dispositions et du régime prévu dans l’accord du 7 décembre 2016 relatif à la mise en place du télétravail applicable au sein de l’entreprise."

Ma réponse :
nombrejourdeteletravailmaxparsemaine=2

Mon raisonnement :
Le document mentionne explicitement que les salariés peuvent télétravailler "deux jours par semaine". Cette information est claire et ne nécessite pas de calcul ou d'hypothèse supplémentaire.

148 T09221026132


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q_TTSEM_NOMBRE:Extraction de deux paragraphes pertinents provenant du document à analyser :

* "Chapitre 1 - Les temps de déconnexion [...] Lorsqu’une tâche ne peut être réalisée à un autre moment que pendant ces périodes de déconnexion ou en cas de circonstances particulières nées de l’urgence et de l’importance des sujets traités, des exceptions à ce principe pourront être admises."
* "Chapitre 2 - Une responsabilité partagée... Cette responsabilité partagée repose sur l’instauration de nouvelles pratiques organisationnelles visant à mieux réguler l’équilibre de nos temps de vie dans un contexte de digitalisation croissante"

Ma réponse :
nombrejourdeteletravailmaxparsemaine=0

Mon raisonnement :
Après avoir analysé les deux paragraphes pertinents du document, je n'ai pas trouvé de mention explicite du nombre de jours de télétravail par semaine. Par conséquent, conformément aux instructions, j'affecte à la variable nombrejourdeteletravailmaxparsemaine la valeur 0.



In [134]:
data_files=glob.glob('*.parquet')
df = pd.concat((pd.read_parquet(f, engine = 'pyarrow') for f in data_files))
df=df.set_index("num_dossier")
df=df[df.nombre_jours_teletravail_semaine.values!=[None]]
df2=pd.read_parquet(f"{PARQUET_FILE_COMPARE}")
df2=df2[["N..Dossier","nombre_jours_teletravail_semaine"]].set_index("N..Dossier")
df_merge=df2.merge(df,how="left",left_index=True, right_index=True)
df_merge.nombre_jours_teletravail_semaine_y=df_merge.nombre_jours_teletravail_semaine_y.astype("float64")
df_merge=df_merge.fillna(0.0)
df_merge["diff"]=(df_merge.nombre_jours_teletravail_semaine_x!=df_merge.nombre_jours_teletravail_semaine_y)
print("global accuracy",1-(df_merge["diff"].sum()/df_merge.shape[0]))

df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x!=0.0]
nb_correct,total =df_rempli.shape[0]-df_rempli["diff"].sum(),df_rempli.shape[0]
print("accuracy on non null value",1-(df_rempli["diff"].sum()/df_rempli.shape[0]),f"; nombre correct : {nb_correct} sur {total}")

df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x==0.0]
nb_correct,total =df_rempli.shape[0]-df_rempli["diff"].sum(),df_rempli.shape[0]
print("accuracy on null value",1-(df_rempli["diff"].sum()/df_rempli.shape[0]),f"; nombre correct : {nb_correct} sur {total}")

global accuracy 0.7152317880794702
accuracy on non null value 0.7708333333333334 ; nombre correct : 74 sur 96
accuracy on null value 0.6181818181818182 ; nombre correct : 34 sur 55


In [135]:
df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x!=0.0]
df_rempli[df_rempli["diff"]]

,nombre_jours_teletravail_semaine_x,nombre_jours_teletravail_semaine_y,diff
N..Dossier,,,
T00621004760,1.0,0.0,True
T01321011053,2.0,1.0,True
T02221003288,2.0,0.0,True
T03121008885,2.0,0.0,True
T03321006914,1.0,0.0,True
T04121001532,5.0,1.0,True
T04521003587,3.0,0.0,True
T05121003353,0.5,0.0,True
T06421003693,2.0,0.0,True
